In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import datasets
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
data = load_dataset( 'HHousen/ParaSCI')
data

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 338717
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 6433
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 4894
    })
})

In [27]:
# data = str(data)

In [31]:
data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 338717
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 6433
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 4894
    })
})

In [32]:
data["train"][0:5]   

{'sentence1': ['for all methods , the tweets were tokenized with the cmu twitter nlp tool .',
  'it was shown by nederhof et al that prefix probabilities can also be effectively computed for probabilistic tree adjoining grammars .',
  'first , kikuchi et al proposed a new long short-term memory network to control the length of the sentence generated by an encoder-decoder model in a text summarization task .',
  'with word confusion networks further improves performance .',
  'fofe can model the word order in a sequence based on a simple ordinally-forgetting mechanism , which uses the position of each word in the sequence .'],
 'sentence2': ['the tweets were tokenized and part-ofspeech tagged with the cmu ark twitter nlp tool and stanford corenlp .',
  'nederhof et al , for instance , show that prefix probabilities , and therefore surprisal , can be estimated from tree adjoining grammars .',
  'first , kikuchi et al tried to control the length of the sentence generated by an encoder-dec

In [35]:
data_string = ""
for s in data.values():
    data_string += s + " "

data_string = " ".join(data_string.values())

data_string

TypeError: unsupported operand type(s) for +: 'Dataset' and 'str'

In [12]:
tokenizer= AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")
tokenizer

BartTokenizerFast(name_or_path='eugenesiow/bart-paraphrase', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained("eugenesiow/bart-paraphrase")
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [16]:
def paraphrase(sent):
    sent = "paraphrase: " + sent + "</s>"
    encoding = tokenizer.encode_plus(sent, padding= True, return_tensors= "pt")
    input_ids = encoding["input_ids"]
    
    outputs = model.generate(
        input_ids = input_ids, 
#         attention_mask = attention_masks, 
        max_length = 256, 
        do_sample = True, 
        top_k = 120, 
        top_p = 0.95, 
        early_stopping = True, 
        num_return_sequences = 1)
    
    output = tokenizer.decode(outputs[0], skip_special_tokens= True, clean_up_tokenization_spaces= True)
    
    return output

In [18]:
output = "".join([paraphrase(sent) for sent in sent_tokenize(data)])


In [19]:
output

"Paraphrase: DatasetDict({\t    _   features: ['sentence1','sentence2'],descriptive : Datasetset({fetishized]])İsfeticism : Phenomena : Phenotype : Fourier : [math]Physics: Phenomenal : Metaphrase : Metasthesis : Metrical : Metropolitan : Metropolis : Metabolic : Meteography : Metes: Meteometer : 4894] Meteometry : Meteleometer : 6433Meteometer"

In [23]:
output = dict(output)

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [22]:
output

['P',
 'a',
 'r',
 'a',
 'p',
 'h',
 'r',
 'a',
 's',
 'e',
 ':',
 ' ',
 'D',
 'a',
 't',
 'a',
 's',
 'e',
 't',
 'D',
 'i',
 'c',
 't',
 '(',
 '{',
 '\t',
 ' ',
 ' ',
 ' ',
 ' ',
 '_',
 ' ',
 ' ',
 ' ',
 'f',
 'e',
 'a',
 't',
 'u',
 'r',
 'e',
 's',
 ':',
 ' ',
 '[',
 "'",
 's',
 'e',
 'n',
 't',
 'e',
 'n',
 'c',
 'e',
 '1',
 "'",
 ',',
 "'",
 's',
 'e',
 'n',
 't',
 'e',
 'n',
 'c',
 'e',
 '2',
 "'",
 ']',
 ',',
 'd',
 'e',
 's',
 'c',
 'r',
 'i',
 'p',
 't',
 'i',
 'v',
 'e',
 ' ',
 ':',
 ' ',
 'D',
 'a',
 't',
 'a',
 's',
 'e',
 't',
 's',
 'e',
 't',
 '(',
 '{',
 'f',
 'e',
 't',
 'i',
 's',
 'h',
 'i',
 'z',
 'e',
 'd',
 ']',
 ']',
 ')',
 'İ',
 's',
 'f',
 'e',
 't',
 'i',
 'c',
 'i',
 's',
 'm',
 ' ',
 ':',
 ' ',
 'P',
 'h',
 'e',
 'n',
 'o',
 'm',
 'e',
 'n',
 'a',
 ' ',
 ':',
 ' ',
 'P',
 'h',
 'e',
 'n',
 'o',
 't',
 'y',
 'p',
 'e',
 ' ',
 ':',
 ' ',
 'F',
 'o',
 'u',
 'r',
 'i',
 'e',
 'r',
 ' ',
 ':',
 ' ',
 '[',
 'm',
 'a',
 't',
 'h',
 ']',
 'P',
 'h',
 'y',
 's',
 'i